<a href="https://colab.research.google.com/github/AI4Finance-LLC/FinRL-Library/blob/master/FinRL_stock_trading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

Tutorials to use OpenAI DRL to trade multiple stocks in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog uses FinRL to reproduce the paper: Practical Deep Reinforcement Learning Approach for Stock Trading, Workshop on Challenges and Opportunities for AI in Financial Services, NeurIPS 2018.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-multiple-stock-trading-7b00763b7530
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [1]:
## install finrl library
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git 'C:\Users\User\AppData\Local\Temp\pip-req-build-mwwbxx1r'
  Running command git clone -q https://github.com/quantopian/pyfolio.git 'C:\Users\User\AppData\Local\Temp\pip-install-nv2ypccs\pyfolio_ad819aacd1a04de490a11f0bc98583c8'
ERROR: Could not install packages due to an OSError: [WinError 5] 拒绝访问。: 'D:\\InstalledSoftware\\anaconda3\\envs\\python-notebook\\Lib\\site-packages\\cv2\\cv2.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to c:\users\user\appdata\local\temp\pip-req-build-mwwbxx1r
  Cloning https://github.com/quantopian/pyfolio.git to c:\users\user\appdata\local\temp\pip-install-nv2ypccs\pyfolio_ad819aacd1a04de490a11f0bc98583c8
  Using cached stockstats-0.3.2-py2.py3-none-any.whl (13 kB)
  Using cached yfinance-0.1.59.tar.gz (25 kB)
  Using cached stable_baselines3-1.0-py3-none-any.whl (152 kB)
  Using cached int_date-0.1.8-py2.py3-none-any.whl (5.0 kB)
  Using cached multitasking-0.0.9.tar.gz (8.1 kB)
  Created wheel for finrl: filename=finrl-0.3.0-py3-none-any.whl size=38613 sha256=3ccf134adc24d48a9065db8565c2bb837b3e8b53677ad20b95d16a848df211f9
  Stored in directory: C:\Users\User\AppData\Local\Temp\pip-ephem-wheel-cache-713fwcwc\wheels\e8\19\74\11261997d6bdca44ba73e8eeedb94a3e3d340259516a0887eb
  Created wheel for pyfolio: filename=pyfolio-0.9.2+75.g4b901f6-py3-none-any.whl size=76260 sha256=8e97991000c289f87defd8fb45d928c8f15ff5a61bb8086


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [8]:
!pip install yfinance
!pip install stockstats
!pip install pyfolio
!pip install gym

  Using cached empyrical-0.5.5-py3-none-any.whl
  Created wheel for pyfolio: filename=pyfolio-0.9.2-py3-none-any.whl size=88681 sha256=886e210cbcecd29e2f020b9273be5ae2a55c524fdc4f55932beccc03daf06864
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\42\16\92\9c9d49efccc497040268960f0825101cc9b1c6e41cdc5a409e
Successfully built pyfolio


In [9]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

D:\InstalledSoftware\anaconda3\envs\python-notebook\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

In [10]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [11]:
# from config.py start_date is a string
config.START_DATE

'2000-01-01'

In [12]:
# from config.py end_date is a string
config.END_DATE

'2021-01-01'

In [13]:
print(config.SP500_20_TICKER)

['AAPL', 'MSFT', 'JPM', 'V', 'RTX', 'PG', 'GS', 'NKE', 'DIS', 'AXP', 'HD', 'INTC', 'WMT', 'IBM', 'MRK', 'UNH', 'KO', 'CAT', 'TRV', 'JNJ', 'CVX', 'MCD', 'VZ', 'CSCO', 'XOM', 'BA', 'MMM', 'PFE', 'WBA', 'DD']


In [14]:
sp500_20_df = YahooDownloader(start_date = config.START_DATE,
                     end_date = config.END_DATE,
                     ticker_list = config.SP500_20_TICKER).fetch_data()

sp_df = YahooDownloader(start_date = config.START_DATE,
                     end_date = config.END_DATE,
                     ticker_list = config.SP500_TICKER).fetch_data()

nas_20_df = YahooDownloader(start_date = config.START_DATE,
                     end_date = config.END_DATE,
                     ticker_list = config.NAS_20_TICKER).fetch_data()
qqq_df = YahooDownloader(start_date = config.START_DATE,
                     end_date = config.END_DATE,
                     ticker_list = config.QQQ_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [15]:
sp500_20_df.head()

(90630, 8)

In [16]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.787006,746015200,AAPL,4
1,2009-01-02,18.570000,19.520000,18.400000,15.698216,10955700,AXP,4
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4
3,2009-01-02,44.910000,46.980000,44.709999,32.830360,7117200,CAT,4
4,2009-01-02,16.410000,17.000000,16.250000,12.592946,40980600,CSCO,4


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [17]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [18]:
processed.shape
processed.head()

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,3.067143,3.251429,3.041429,2.787006,746015200,AAPL,4,0.0,3.012162,2.679473,100.0,66.666667,100.0,2.787006,2.787006,0.0
1,2009-01-02,18.570000,19.520000,18.400000,15.698216,10955700,AXP,4,0.0,3.012162,2.679473,100.0,66.666667,100.0,15.698216,15.698216,0.0
2,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4,0.0,3.012162,2.679473,100.0,66.666667,100.0,33.941101,33.941101,0.0
3,2009-01-02,44.910000,46.980000,44.709999,32.830360,7117200,CAT,4,0.0,3.012162,2.679473,100.0,66.666667,100.0,32.830360,32.830360,0.0
4,2009-01-02,16.410000,17.000000,16.250000,12.592946,40980600,CSCO,4,0.0,3.012162,2.679473,100.0,66.666667,100.0,12.592946,12.592946,0.0


In [19]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])
processed_full = processed_full.fillna(0)

In [20]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.787006,746015200.0,4.0,0.0,3.012162,2.679473,100.0,66.666667,100.0,2.787006,2.787006,0.0
1,2009-01-02,AXP,18.570000,19.520000,18.400000,15.698216,10955700.0,4.0,0.0,3.012162,2.679473,100.0,66.666667,100.0,15.698216,15.698216,0.0
2,2009-01-02,BA,42.799999,45.560001,42.779999,33.941101,7010200.0,4.0,0.0,3.012162,2.679473,100.0,66.666667,100.0,33.941101,33.941101,0.0
3,2009-01-02,CAT,44.910000,46.980000,44.709999,32.830360,7117200.0,4.0,0.0,3.012162,2.679473,100.0,66.666667,100.0,32.830360,32.830360,0.0
4,2009-01-02,CSCO,16.410000,17.000000,16.250000,12.592946,40980600.0,4.0,0.0,3.012162,2.679473,100.0,66.666667,100.0,12.592946,12.592946,0.0
5,2009-01-02,CVX,74.230003,77.300003,73.580002,47.373642,13695900.0,4.0,0.0,3.012162,2.679473,100.0,66.666667,100.0,47.373642,47.373642,0.0
6,2009-01-02,DD,21.605234,22.060680,20.993229,15.336650,13251037.0,4.0,0.0,3.012162,2.679473,100.0,66.666667,100.0,15.336650,15.336650,0.0
7,2009-01-02,DIS,22.760000,24.030001,22.500000,20.597496,9796600.0,4.0,0.0,3.012162,2.679473,100.0,66.666667,100.0,20.597496,20.597496,0.0
8,2009-01-02,GS,84.019997,87.620003,82.190002,72.559456,14088500.0,4.0,0.0,3.012162,2.679473,100.0,66.666667,100.0,72.559456,72.559456,0.0
9,2009-01-02,HD,23.070000,24.190001,22.959999,17.797922,14902500.0,4.0,0.0,3.012162,2.679473,100.0,66.666667,100.0,17.797922,17.797922,0.0


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2018-12-31
## Trade data split: 2019-01-01 to 2020-09-30

In [21]:
train = data_split(processed_full, '2009-01-01','2019-01-01')
trade = data_split(processed_full, '2019-01-01','2021-01-01')
print(len(train))
print(len(trade))

75480
15150


In [22]:
initialData = train.loc[0,:]
print(initialData)

         date   tic       open       high        low      close       volume  \
0  2009-01-02  AAPL   3.067143   3.251429   3.041429   2.787006  746015200.0   
0  2009-01-02   AXP  18.570000  19.520000  18.400000  15.698216   10955700.0   
0  2009-01-02    BA  42.799999  45.560001  42.779999  33.941101    7010200.0   
0  2009-01-02   CAT  44.910000  46.980000  44.709999  32.830360    7117200.0   
0  2009-01-02  CSCO  16.410000  17.000000  16.250000  12.592946   40980600.0   
0  2009-01-02   CVX  74.230003  77.300003  73.580002  47.373642   13695900.0   
0  2009-01-02    DD  21.605234  22.060680  20.993229  15.336650   13251037.0   
0  2009-01-02   DIS  22.760000  24.030001  22.500000  20.597496    9796600.0   
0  2009-01-02    GS  84.019997  87.620003  82.190002  72.559456   14088500.0   
0  2009-01-02    HD  23.070000  24.190001  22.959999  17.797922   14902500.0   
0  2009-01-02   IBM  83.889999  87.589996  83.889999  59.180256    7558200.0   
0  2009-01-02  INTC  14.690000  15.25000

In [23]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2019-01-02,AAPL,38.722500,39.712502,38.557499,38.439735,148158800.0,2.0,-2.013469,44.430054,35.384485,37.867359,-91.575723,42.250808,41.155815,46.409359,51.409187
0,2019-01-02,AXP,93.910004,96.269997,93.769997,92.042946,4175400.0,2.0,-3.403805,109.907410,84.174485,41.204998,-97.759092,26.709417,99.719922,100.286694,51.409187
0,2019-01-02,BA,316.190002,323.950012,313.709991,314.645142,3292200.0,2.0,-5.550592,339.116375,287.252015,47.010000,-21.712382,13.611972,314.427160,331.956148,51.409187
0,2019-01-02,CAT,124.029999,127.879997,123.000000,118.137177,4783200.0,2.0,-0.680049,125.145761,109.015061,48.229091,-5.147002,0.873482,117.640500,119.834356,51.409187
0,2019-01-02,CSCO,42.279999,43.200001,42.209999,39.772106,23833500.0,2.0,-0.945558,46.120801,36.852423,44.872560,-87.538213,29.529377,41.863034,42.137596,51.409187


In [24]:
config.TECHNICAL_INDICATORS_LIST

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [25]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 30, State Space: 301


In [26]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

[1000000, 2.78700590133667, 15.698216438293457, 33.94110107421875, 32.830360412597656, 12.59294605255127, 47.37364196777344, 15.336649894714355, 20.597496032714844, 72.55945587158203, 17.797922134399414, 59.18025588989258, 10.399736404418945, 42.00360870361328, 23.343862533569336, 15.628469467163086, 44.31218719482422, 42.266517639160156, 19.942874908447266, 15.435609817504883, 11.325711250305176, 10.71481990814209, 42.82729721069336, 25.695541381835938, 33.274017333984375, 23.23461151123047, 12.23261833190918, 17.803312301635742, 19.09723472595215, 42.599510192871094, 51.8355827331543, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121

## Environment for Training



In [27]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

[1000000, 2.78700590133667, 15.698216438293457, 33.94110107421875, 32.830360412597656, 12.59294605255127, 47.37364196777344, 15.336649894714355, 20.597496032714844, 72.55945587158203, 17.797922134399414, 59.18025588989258, 10.399736404418945, 42.00360870361328, 23.343862533569336, 15.628469467163086, 44.31218719482422, 42.266517639160156, 19.942874908447266, 15.435609817504883, 11.325711250305176, 10.71481990814209, 42.82729721069336, 25.695541381835938, 33.274017333984375, 23.23461151123047, 12.23261833190918, 17.803312301635742, 19.09723472595215, 42.599510192871094, 51.8355827331543, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [28]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [29]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [30]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=10)

[1000000, 2.78700590133667, 15.698216438293457, 33.94110107421875, 32.830360412597656, 12.59294605255127, 47.37364196777344, 15.336649894714355, 20.597496032714844, 72.55945587158203, 17.797922134399414, 59.18025588989258, 10.399736404418945, 42.00360870361328, 23.343862533569336, 15.628469467163086, 44.31218719482422, 42.266517639160156, 19.942874908447266, 15.435609817504883, 11.325711250305176, 10.71481990814209, 42.82729721069336, 25.695541381835938, 33.274017333984375, 23.23461151123047, 12.23261833190918, 17.803312301635742, 19.09723472595215, 42.599510192871094, 51.8355827331543, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121

### Model 2: DDPG

In [31]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [32]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

[1000000, 2.5284206867218018, 13.93314266204834, 31.848371505737305, 27.9780216217041, 11.746487617492676, 44.41432189941406, 14.95845890045166, 18.479190826416016, 61.09346008300781, 17.111970901489258, 57.520748138427734, 9.400814056396484, 39.78050994873047, 16.99224281311035, 14.9270601272583, 41.4761962890625, 40.28850555419922, 18.135156631469727, 14.964872360229492, 9.989506721496582, 10.263236999511719, 39.369720458984375, 23.895219802856445, 30.21897315979004, 21.474538803100586, 10.852323532104492, 15.622865676879883, 20.113765716552734, 38.412567138671875, 49.58792495727539, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.030929269621033484, -0.23612444797124077, -0.3150872449937907, -0.4335741830174662, -0.12089282699944626, -0.3251487333679819, -0.060014613611418355, -0.21986939663733196, -1.2200046143570091, -0.14673268084856872, -0.23615532837777664, -0.10919255846630094, -0.16495318449447183, -0.43376620256959697, -0.06730178

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

day: 2515, episode: 10
begin_total_asset: 1000000.00
end_total_asset: 2977875.91
total_reward: 1977875.91
total_cost: 1070.38
total_trades: 35258
Sharpe: 0.683
[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321

...

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

day: 2515, episode: 20
begin_total_asset: 1000000.00
end_total_asset: 2977875.91
total_reward: 1977875.91
total_cost: 1070.38
total_trades: 35258
Sharpe: 0.683
[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

### Model 3: PPO

In [33]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


In [34]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

[1000000, 2.78700590133667, 15.698216438293457, 33.94110107421875, 32.830360412597656, 12.59294605255127, 47.37364196777344, 15.336649894714355, 20.597496032714844, 72.55945587158203, 17.797922134399414, 59.18025588989258, 10.399736404418945, 42.00360870361328, 23.343862533569336, 15.628469467163086, 44.31218719482422, 42.266517639160156, 19.942874908447266, 15.435609817504883, 11.325711250305176, 10.71481990814209, 42.82729721069336, 25.695541381835938, 33.274017333984375, 23.23461151123047, 12.23261833190918, 17.803312301635742, 19.09723472595215, 42.599510192871094, 51.8355827331543, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121

-----------------------------------------
| environment/            |             |
|    portfolio_value      | 3.24e+06    |
|    total_cost           | 2.8e+05     |
|    total_reward         | 2.24e+06    |
|    total_reward_pct     | 224         |
|    total_trades         | 73342       |
| time/                   |             |
|    fps                  | 156         |
|    iterations           | 2           |
|    time_elapsed         | 26          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014255486 |
|    clip_fraction        | 0.216       |
|    clip_range           | 0.2         |
|    entropy_loss         | -42.6       |
|    explained_variance   | -0.0121     |
|    learning_rate        | 0.00025     |
|    loss                 | 5.85        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0249     |
|    std                  | 1           |
|    value_loss           | 14.6  

-----------------------------------------
| environment/            |             |
|    portfolio_value      | 2.56e+06    |
|    total_cost           | 2.69e+05    |
|    total_reward         | 1.56e+06    |
|    total_reward_pct     | 156         |
|    total_trades         | 72108       |
| time/                   |             |
|    fps                  | 152         |
|    iterations           | 4           |
|    time_elapsed         | 53          |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.009366335 |
|    clip_fraction        | 0.227       |
|    clip_range           | 0.2         |
|    entropy_loss         | -42.7       |
|    explained_variance   | -0.0214     |
|    learning_rate        | 0.00025     |
|    loss                 | 5.04        |
|    n_updates            | 30          |
|    policy_gradient_loss | -0.0292     |
|    std                  | 1.01        |
|    value_loss           | 14.7  

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

----------------------------------------
| environment/            |            |
|    portfolio_value      | 3.23e+06   |
|    total_cost           | 2.65e+05   |
|    total_reward         | 2.23e+06   |
|    total_reward_pct     | 223        |
|    total_trades         | 71914      |
| time/                   |            |
|    fps                  | 152        |
|    iterations           | 8          |
|    time_elapsed         | 107        |
|    total_timesteps      | 16384      |
| train/                  |            |
|    approx_kl            | 0.01284576 |
|    clip_fraction        | 0.181      |
|    clip_range           | 0.2        |
|    entropy_loss         | -42.9      |
|    explained_variance   | -0.00645   |
|    learning_rate        | 0.00025    |
|    loss                 | 7.57       |
|    n_updates            | 70         |
|    policy_gradient_loss | -0.0222    |
|    std                  | 1.01       |
|    value_loss           | 17.9       |
----------------

...

----------------------------------------
| environment/            |            |
|    portfolio_value      | 3.05e+06   |
|    total_cost           | 2.63e+05   |
|    total_reward         | 2.05e+06   |
|    total_reward_pct     | 205        |
|    total_trades         | 71392      |
| time/                   |            |
|    fps                  | 150        |
|    iterations           | 16         |
|    time_elapsed         | 217        |
|    total_timesteps      | 32768      |
| train/                  |            |
|    approx_kl            | 0.03348986 |
|    clip_fraction        | 0.242      |
|    clip_range           | 0.2        |
|    entropy_loss         | -43.1      |
|    explained_variance   | 0.00483    |
|    learning_rate        | 0.00025    |
|    loss                 | 8.57       |
|    n_updates            | 150        |
|    policy_gradient_loss | -0.0167    |
|    std                  | 1.02       |
|    value_loss           | 17.5       |
----------------

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

-----------------------------------------
| environment/            |             |
|    portfolio_value      | 2.88e+06    |
|    total_cost           | 2.53e+05    |
|    total_reward         | 1.88e+06    |
|    total_reward_pct     | 188         |
|    total_trades         | 70189       |
| time/                   |             |
|    fps                  | 149         |
|    iterations           | 20          |
|    time_elapsed         | 273         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.027499339 |
|    clip_fraction        | 0.255       |
|    clip_range           | 0.2         |
|    entropy_loss         | -43.3       |
|    explained_variance   | 0.00184     |
|    learning_rate        | 0.00025     |
|    loss                 | 9.09        |
|    n_updates            | 190         |
|    policy_gradient_loss | -0.0175     |
|    std                  | 1.03        |
|    value_loss           | 24    

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

-----------------------------------------
| environment/            |             |
|    portfolio_value      | 2.28e+06    |
|    total_cost           | 2.57e+05    |
|    total_reward         | 1.28e+06    |
|    total_reward_pct     | 128         |
|    total_trades         | 70530       |
| time/                   |             |
|    fps                  | 149         |
|    iterations           | 24          |
|    time_elapsed         | 328         |
|    total_timesteps      | 49152       |
| train/                  |             |
|    approx_kl            | 0.028221237 |
|    clip_fraction        | 0.26        |
|    clip_range           | 0.2         |
|    entropy_loss         | -43.5       |
|    explained_variance   | -0.0545     |
|    learning_rate        | 0.00025     |
|    loss                 | 9.61        |
|    n_updates            | 230         |
|    policy_gradient_loss | -0.0219     |
|    std                  | 1.03        |
|    value_loss           | 22.7  

### Model 4: TD3

In [35]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [36]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

[1000000, 17.935117721557617, 50.988304138183594, 61.513343811035156, 65.38973999023438, 12.984989166259766, 73.7756118774414, 34.809871673583984, 43.42416763305664, 83.53462219238281, 43.64031982421875, 141.16954040527344, 20.43437957763672, 52.85589599609375, 27.817968368530273, 29.65427017211914, 68.79739379882812, 70.55941009521484, 31.62615394592285, 25.23847007751465, 19.80267906188965, 15.745709419250488, 46.61450958251953, 38.54338455200195, 51.902278900146484, 50.927310943603516, 29.01645851135254, 29.77606964111328, 23.699634552001953, 56.53315353393555, 59.16899871826172, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.048914665320310746, 0.1492300754165683, 0.22427827362162844, -1.7378322917263773, -0.07238045700446705, 0.4739803415935455, -0.04486043243978344, 0.7491596831102143, -1.3622740921689882, 0.48416460733987776, -0.6495137919078786, -0.04034402844369822, 0.8152996071794831, 0.052010051080728203, 0.21119042031527968, -0.4

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

day: 2515, episode: 50
begin_total_asset: 1000000.00
end_total_asset: 3787567.16
total_reward: 2787567.16
total_cost: 999.00
total_trades: 42755
Sharpe: 0.808
[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321,

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

### Model 5: SAC

In [37]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 1000000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [ ]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=80000)

[1000000, 2.78700590133667, 15.698216438293457, 33.94110107421875, 32.830360412597656, 12.59294605255127, 47.37364196777344, 15.336649894714355, 20.597496032714844, 72.55945587158203, 17.797922134399414, 59.18025588989258, 10.399736404418945, 42.00360870361328, 23.343862533569336, 15.628469467163086, 44.31218719482422, 42.266517639160156, 19.942874908447266, 15.435609817504883, 11.325711250305176, 10.71481990814209, 42.82729721069336, 25.695541381835938, 33.274017333984375, 23.23461151123047, 12.23261833190918, 17.803312301635742, 19.09723472595215, 42.599510192871094, 51.8355827331543, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121616168871115, 3.0121

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

[1000000, 38.39591979980469, 91.69662475585938, 313.3722229003906, 118.78218841552734, 40.123985290527344, 97.55754852294922, 72.68486785888672, 108.32612609863281, 159.20101928710938, 162.39141845703125, 100.57974243164062, 44.13541793823242, 121.43285369873047, 90.1629638671875, 44.01080322265625, 168.1305389404297, 176.2595672607422, 71.46668243408203, 98.81327819824219, 72.5230484008789, 37.58075714111328, 86.1761703491211, 63.318092346191406, 113.30058288574219, 240.15750122070312, 129.90611267089844, 50.585201263427734, 62.84025573730469, 89.07861328125, 58.40364074707031, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.141041777965299, -3.552996723704098, -6.5322711673567255, -0.8822167825157834, -0.9799449674846201, -2.1934933168446946, -1.5881746002519037, -1.8656878145195321, -9.081052234113486, -2.2328399684980695, -2.5064249292951075, -0.32983955379442875, -4.102228793422313, -2.889179649527293, -0.37578329318664316, -1.582194261

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
data_turbulence = processed_full[(processed_full.date<'2019-01-01') & (processed_full.date>='2009-01-01')]
insample_turbulence = data_turbulence.drop_duplicates(subset=['date'])

In [ ]:
insample_turbulence.turbulence.describe()

In [ ]:
turbulence_threshold = np.quantile(insample_turbulence.turbulence.values,1)

In [ ]:
turbulence_threshold

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2018-12 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
trade = data_split(processed_full, '2019-01-01','2021-01-01')
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 380, **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trade.head()

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.tail()

In [ ]:
df_actions.head()

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = '2019-01-01',
        end = '2021-01-01')

stats = backtest_stats(baseline_df, value_col_name = 'close')


<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = '2019-01-01',
             baseline_end = '2021-01-01')